In [1]:
import h5py
import numpy as np
import os.path
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

## Read HDF5 file

In [2]:
f = h5py.File("fileout.h5", "r")
for key in f.keys():
    print(key, ":", type(f[key])) #Names of the root level object names in HDF5 file - can be groups or datasets.

B0x : <class 'h5py._hl.dataset.Dataset'>
B0y : <class 'h5py._hl.dataset.Dataset'>
B0z : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint00160 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint00320 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint00480 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint00640 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint00800 : <class 'h5py._hl.dataset.Dataset'>
config : <class 'h5py._hl.group.Group'>
detAnt_01 : <class 'h5py._hl.group.Group'>
detAnt_02 : <class 'h5py._hl.group.Group'>
n_e : <class 'h5py._hl.dataset.Dataset'>


In [3]:
for key in f["detAnt_01"].keys():
    print(key, ":",type(f["detAnt_01"][key])) #Names of the root level object names in HDF5 file - can be groups or datasets.

detAnt_ypos : <class 'h5py._hl.dataset.Dataset'>
detAnt_zpos : <class 'h5py._hl.dataset.Dataset'>
i : <class 'h5py._hl.dataset.Dataset'>
rms_E : <class 'h5py._hl.dataset.Dataset'>
sum_EE : <class 'h5py._hl.dataset.Dataset'>
sum_ExEx : <class 'h5py._hl.dataset.Dataset'>
sum_EyEy : <class 'h5py._hl.dataset.Dataset'>
sum_EzEz : <class 'h5py._hl.dataset.Dataset'>


## Plot data HDF5: Plasma density

In [4]:
density = f['E_abs__tint00320'][:]
density = np.where(density < 0.01, 0, density)

In [5]:
print(f"Shape: {density.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {density.dtype}")  # Data type of the elements
print(f"Size: {density.size}")  # Total number of elements in the dataset

Shape: (200, 150, 100)
Data Type (dtype): float64
Size: 3000000


In [6]:
# Generate the coordinate grid for the 3D volume (X, Y, Z)
x = density.shape[0]
y = density.shape[1]
z = density.shape[2]
X, Y, Z = np.mgrid[ 0:x:x*1j, 0:y:y*1j, 0:z:z*1j ]

In [7]:
# Create a 3D Volume Plot using Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),                 # X-coordinates
    y=Y.flatten(),                 # Y-coordinates
    z=Z.flatten(),                 # Z-coordinates
    value = density.flatten(),     # Flattened volume data
    isomin=0.01,                    # Minimum value to display
    isomax=np.max(density),        # Maximum value to display
    opacity=0.2,                   # Adjust opacity (between 0 and 1) for better visualization
    surface_count=5,              # Number of volume surfaces to display
    colorscale='plasma',           # Choose a colorscale
))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    zaxis_title='Z Axis'
))

# Show the plot
fig.show()

## Plot data HDF5: Electric field

In [7]:
E_field_ds = f['E_abs__tint00160'][:]
print(f"Shape: {E_field_ds.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {E_field_ds.dtype}")  # Data type of the elements
print(f"Size: {E_field_ds.size}")  # Total number of elements in the dataset

Shape: (200, 150, 30)
Data Type (dtype): float64
Size: 900000


In [8]:
# Create a 3D Volume Plot using Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),                    # X-coordinates
    y=Y.flatten(),                    # Y-coordinates
    z=Z.flatten(),                    # Z-coordinates
    value = E_field_ds.flatten(),     # Flattened volume data
    isomin=0.5, #-np.max(E_field_ds),       # Minimum value to display
    isomax=np.max(E_field_ds),        # Maximum value to display
    opacity=0.2,                      # Adjust opacity (between 0 and 1) for better visualization
    surface_count=10,                 # Number of volume surfaces to display
    colorscale='jet',                 # Choose a colorscale
))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    zaxis_title='Z Axis'
))

# Show the plot
fig.show()

## Magnetic field

In [16]:
B_field_ds = E_f['B_abs'][:]
print(f"Shape: {B_field_ds.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {B_field_ds.dtype}")  # Data type of the elements
print(f"Size: {B_field_ds.size}")  # Total number of elements in the dataset

Shape: (150, 150, 200)
Data Type (dtype): float64
Size: 4500000


In [17]:
# Create a 3D Volume Plot using Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),                 # X-coordinates
    y=Y.flatten(),                 # Y-coordinates
    z=Z.flatten(),                 # Z-coordinates
    value = B_field_ds.flatten(),     # Flattened volume data
    isomin=0.2,                    # Minimum value to display
    isomax=np.max(B_field_ds),        # Maximum value to display
    opacity=0.3,                   # Adjust opacity (between 0 and 1) for better visualization
    surface_count=10,              # Number of volume surfaces to display
    colorscale='plasma',           # Choose a colorscale
))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    zaxis_title='Z Axis'
))

# Show the plot
fig.show()